In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import opendatasets as od
import tensorflow as tf
import cv2
from tqdm import *

In [2]:
dataset_directory = "C:\\Users\\amith\\Documents\\Datasets\\kaggle\\kaggle".replace("\\","/")
csv_directory = "C:\\Users\\amith\\Documents\\Datasets".replace("\\","/")

In [3]:
train_val_age = pd.read_csv(csv_directory + "/train_age.csv").sort_values(by="imageId")
train_val_gender = pd.read_csv(csv_directory + "/train_gender.csv").sort_values(by="imageId")

id = train_val_age["imageId"].values.tolist()
train_val_files = pd.DataFrame(columns=["imageId","Files"])
ctr = 0

for i in os.listdir(dataset_directory+"/train"):
    train_val_files.loc[ctr,"imageId"] = id[ctr]
    train_val_files.loc[ctr,"Files"] = i
    ctr += 1

train_val_files = train_val_files.sort_values(by="imageId")

train_val_output = pd.merge(left=train_val_age,right=train_val_gender,on="imageId",how="inner")
train_val_output = pd.merge(left=train_val_output,right=train_val_files,on="imageId",how="inner")

train_val_output = train_val_output.sort_values(by="imageId")

train_val_id = train_val_output["imageId"].values.tolist()
np.random.shuffle(train_val_id)


threshold = int(np.round(0.8 * len(train_val_id)))

train_id = train_val_id[0:threshold]
val_id = train_val_id[threshold:]

train_val_output.set_index(keys="imageId",drop=False,inplace=True)

train_output = train_val_output.loc[train_id,:]
val_output = train_val_output.loc[val_id,:]

train_output = train_output.reset_index(drop=True).drop(columns=["imageId"])
val_output = val_output.reset_index(drop=True).drop(columns=["imageId"])

In [4]:
train_output.head()

,age,gender,Files
0,38.0,1,008135.png
1,54.0,1,009042.png
2,41.0,1,008853.png
3,46.0,0,000890.png
4,46.0,0,004888.png


In [5]:
val_output.head()

,age,gender,Files
0,31.0,0,009312.png
1,23.0,0,009787.png
2,23.0,1,000927.png
3,29.0,1,002109.png
4,60.0,1,002944.png


In [6]:
train_images = []
train_data = {}
ctr = 0
for i in tqdm(iterable=train_output["Files"],desc="Processing train data"):
    ages = train_output.loc[ctr,"age"]
    genders = train_output.loc[ctr,"gender"]
    img_org = cv2.imread(dataset_directory + "/train/" + i)
    img_gray = cv2.cvtColor(src=img_org, code=cv2.COLOR_BGR2GRAY)
    img_short = cv2.resize(src=img_gray, dsize=(64,64))
    img = img_short / 255.0
    train_images.append(img)
    ctr += 1

train_images = np.array(train_images)

Processing train data: 100%|██████████| 8562/8562 [05:10<00:00, 27.61it/s]


In [7]:
dataset_directory + "/" + i

'C:/Users/amith/Documents/Datasets/kaggle/kaggle/009961.png'

In [8]:
dataset_directory + "/val/" + i

'C:/Users/amith/Documents/Datasets/kaggle/kaggle/val/009961.png'

In [9]:
val_images = []
val_data = {}
ctr = 0
for i in tqdm(iterable=val_output["Files"],desc="Processing val data"):
    ages = val_output.loc[ctr,"age"]
    genders = val_output.loc[ctr,"gender"]
    img_org = cv2.imread(dataset_directory + "/train/" + i)
    img_gray = cv2.cvtColor(src=img_org, code=cv2.COLOR_BGR2GRAY)
    img_short = cv2.resize(src=img_gray, dsize=(64,64))
    img = img_short / 255.0
    val_images.append(img)
    ctr += 1

val_images = np.array(val_images)

Processing val data: 100%|██████████| 2140/2140 [01:38<00:00, 21.68it/s]


In [10]:
train_images = train_images.reshape(tuple(list(train_images.shape) + [1]))
val_images = val_images.reshape(tuple(list(val_images.shape) + [1]))

In [11]:
val_images.shape

(2140, 64, 64, 1)

In [12]:
model_genders = tf.keras.models.Sequential(layers=[
    tf.keras.layers.Conv2D(filters=16,kernel_size=(3,3),activation="relu",input_shape=(64,64,1)),
    tf.keras.layers.Conv2D(filters=32,kernel_size=(3,3),activation="relu",input_shape=(64,64,1)),
    tf.keras.layers.Conv2D(filters=64,kernel_size=(3,3),activation="relu",input_shape=(64,64,1)),
    tf.keras.layers.Conv2D(filters=64,kernel_size=(3,3),activation="relu",input_shape=(64,64,1)),
    
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(units=64,activation="relu"),
    tf.keras.layers.Dense(units=1,activation="softmax")
])

In [13]:
model_ages = tf.keras.models.Sequential(layers=[
    tf.keras.layers.Conv2D(filters=16,kernel_size=(3,3),activation="relu",input_shape=(64,64,1)),
    tf.keras.layers.Conv2D(filters=32,kernel_size=(3,3),activation="relu",input_shape=(64,64,1)),
    tf.keras.layers.Conv2D(filters=64,kernel_size=(3,3),activation="relu",input_shape=(64,64,1)),
    tf.keras.layers.Conv2D(filters=64,kernel_size=(3,3),activation="relu",input_shape=(64,64,1)),

    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(units=64,activation="relu"),
    tf.keras.layers.Dense(units=1,activation="linear")
])

In [14]:
train_ages = train_output["age"].values
train_genders = train_output["gender"].values

val_ages = val_output["age"].values
val_genders = val_output["gender"].values

In [15]:
model_genders.compile(optimizer=tf.keras.optimizers.Adam(),loss=tf.keras.losses.binary_crossentropy,metrics=['accuracy'])
model_ages.compile(optimizer=tf.keras.optimizers.SGD(),loss=tf.keras.losses.mean_absolute_error, metrics=["mae"])

In [16]:
model_genders.fit(x=train_images,y=train_genders,batch_size=32,epochs=10,validation_data=(val_images,val_genders),verbose=1,shuffle=True,callbacks=tf.keras.callbacks.EarlyStopping(patience=2))

Epoch 1/10
268/268 [==============================] - 91s 333ms/step - loss: 0.4336 - accuracy: 0.4178 - val_loss: 0.3295 - val_accuracy: 0.4280
Epoch 2/10
268/268 [==============================] - 65s 242ms/step - loss: 0.3024 - accuracy: 0.4178 - val_loss: 0.3224 - val_accuracy: 0.4280
Epoch 3/10
268/268 [==============================] - 51s 189ms/step - loss: 0.2502 - accuracy: 0.4178 - val_loss: 0.2915 - val_accuracy: 0.4280
Epoch 4/10
268/268 [==============================] - 49s 183ms/step - loss: 0.2031 - accuracy: 0.4178 - val_loss: 0.2481 - val_accuracy: 0.4280
Epoch 5/10
268/268 [==============================] - 116s 433ms/step - loss: 0.1669 - accuracy: 0.4178 - val_loss: 0.2135 - val_accuracy: 0.4280
Epoch 6/10
268/268 [==============================] - 79s 297ms/step - loss: 0.1260 - accuracy: 0.4178 - val_loss: 0.2738 - val_accuracy: 0.4280
Epoch 7/10
268/268 [==============================] - 138s 514ms/step - loss: 0.0985 - accuracy: 0.4178 - val_loss: 0.2551 - val_

In [17]:
model_ages.fit(x=train_images,y=train_ages,batch_size=32,epochs=10,validation_data=(val_images,val_ages),verbose=1,shuffle=True,callbacks=tf.keras.callbacks.EarlyStopping(patience=2))

Epoch 1/10
268/268 [==============================] - 112s 414ms/step - loss: 27.9639 - mae: 27.9639 - val_loss: 17.4535 - val_mae: 17.4535
Epoch 2/10
268/268 [==============================] - 148s 553ms/step - loss: 22.5641 - mae: 22.5641 - val_loss: 42.5140 - val_mae: 42.5140
Epoch 3/10
268/268 [==============================] - 35s 130ms/step - loss: 22.8956 - mae: 22.8956 - val_loss: 44.6128 - val_mae: 44.6128


In [20]:
tf.keras.models.save_model(model=model_genders,filepath="C:/Users/amith/Documents/Datasets/model_genders.h5",overwrite=True)
tf.keras.models.save_model(model=model_ages,filepath="C:/Users/amith/Documents/Datasets/model_ages.h5",overwrite=True)

C:\Users\amith\AppData\Local\Temp\ipykernel_44388\4190216403.py:1: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  tf.keras.models.save_model(model=model_genders,filepath="C:/Users/amith/Documents/Datasets/model_genders.h5",overwrite=True)
C:\Users\amith\AppData\Local\Temp\ipykernel_44388\4190216403.py:2: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  tf.keras.models.save_model(model=model_ages,filepath="C:/Users/amith/Documents/Datasets/model_ages.h5",overwrite=True)


In [21]:
print(dataset_directory + "/test/")

C:/Users/amith/Documents/Datasets/kaggle/kaggle/test/


In [22]:
test_images = []

ctr = 0
for i in tqdm(iterable=sorted(os.listdir(dataset_directory + "/test/")),desc="Processing test data"):
    img_org = cv2.imread(dataset_directory + "/test/" + i)
    img_gray = cv2.cvtColor(src=img_org, code=cv2.COLOR_BGR2GRAY)
    img_short = cv2.resize(src=img_gray, dsize=(64,64))
    img = img_short / 255.0
    test_images.append(img)
    ctr += 1

test_images = np.array(test_images)

Processing test data: 100%|██████████| 11747/11747 [06:00<00:00, 32.55it/s]


In [31]:
test_ages = pd.read_csv("C:\\Users\\amith\\Documents\\Datasets\\sample_submission_age.csv")["age"].copy()
test_genders = pd.read_csv("C:\\Users\\amith\\Documents\\Datasets\\sample_submission_gender.csv")["gender"].copy()

In [25]:
predict_ages = model_ages.predict(test_images)

368/368 [==============================] - 13s 36ms/step


In [28]:
predict_ages.min()

95.05065

In [34]:
test_ages.min()

0